# [Transit Engine](https://developer.hashicorp.com/vault/api-docs/secret/transit)

In [1]:
%env  WORKDIR=/tmp/vault    

env: WORKDIR=/tmp/vault


In [2]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

hvs.QbtInWOOYqrZJnMrDmcaArKs
https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200


In [3]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs

env: VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
env: VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs


In [4]:
%%bash
vault secrets list -detailed

Path          Plugin       Accessor              Default TTL    Max TTL    Force No Cache    Replication    Seal Wrap    External Entropy Access    Options           Description                                                UUID                                    Version    Running Version          Running SHA256    Deprecation Status
----          ------       --------              -----------    -------    --------------    -----------    ---------    -----------------------    -------           -----------                                                ----                                    -------    ---------------          --------------    ------------------
cubbyhole/    cubbyhole    cubbyhole_9ca6d24d    n/a            n/a        false             local          false        false                      map[]             per-token private secret storage                           4951a408-65bd-0832-e327-453178964d39    n/a        v1.18.1+builtin.vault    n/a               n/a
d

In [5]:
%%bash
# Enable Transit
vault secrets enable transit

Success! Enabled the transit secrets engine at: transit/


## Encryption

In [6]:
%%bash
# Tipos de kes soportados: https://developer.hashicorp.com/vault/api-docs/secret/transit#type
vault write -f transit/keys/orders

Key                       Value
---                       -----
allow_plaintext_backup    false
auto_rotate_period        0s
deletion_allowed          false
derived                   false
exportable                false
imported_key              false
keys                      map[1:1731938662]
latest_version            1
min_available_version     0
min_decryption_version    1
min_encryption_version    0
name                      orders
supports_decryption       true
supports_derivation       true
supports_encryption       true
supports_signing          false
type                      aes256-gcm96


In [7]:
%%bash
# ENCRYPT
vault write transit/encrypt/orders plaintext=$(base64 <<< "4111 1111 1111 1111") -format=json | jq -r .data.ciphertext

export CIPHERTEXT=$(vault write transit/encrypt/orders plaintext=$(base64 <<< "4111 1111 1111 1111") -format=json | jq -r .data.ciphertext)

# DECRYPT
vault write transit/decrypt/orders ciphertext=$CIPHERTEXT -format=json | jq -r .data.plaintext | base64 -d


vault:v1:oHd+WuM9s7mWWnfWfl+ph/VfNDBiSFN39GbaE6Cfw+9F8/kr9nNru9cYE54LpAF2
4111 1111 1111 1111


## Sign

In [8]:
%%bash
vault write -f  transit/keys/signature type=rsa-4096

Key                       Value
---                       -----
allow_plaintext_backup    false
auto_rotate_period        0s
deletion_allowed          false
derived                   false
exportable                false
imported_key              false
keys                      map[1:map[certificate_chain: creation_time:2024-11-18T14:05:23.014923344Z name:rsa-4096 public_key:-----BEGIN PUBLIC KEY-----
MIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEAyd7Mi5uz0DQcOKOV2KFI
V2BheHZfujWzHd5lrirG5VamxKgQfJCYupeENfUKDWadreWbyESq80umVlDRkNEI
8+PHyTrbPWLpmCs/FI1FwnwtPb3tyUZv+G3tdzrGg7L6kjdAwS0df5KpEy1eQEMa
A+Qn3DUyw1m62TvHXgafx/wrR5tGG4QczrrMS8YpLAPBLI7llFo0mIB9wU2qSgnA
ffdPGR1zJQ8xmzvR7xVZyXqxErD+sAXOAJ1ez5jergVEQJ9PQHO2rQ8G0F0Y4g/4
fSSwYg9KYvXL1/zvAzt1tv5CREpi8g/kmrayDUneSG3Ee/8a3n3m+Wg39OlUK9Br
+vPQrtZzSDbI1RQQlFF4nG49RTxnCTr8hmQPivfObJn6Uni5ED5nGh19wWrH0ohl
U7k888aYHRVySWjoiU98SsBCVYIU4g65Rxrzfv8QhNo1pyIOPwkUmNr3IMYTqEnI
JYlAtUjJOhrqpMFkzrU/a1c6y2pP9GrFeHFVn+7mOdegZsLZjNVvm8RCr01QZkNY
DtdNFD0lFf

In [9]:
%%bash
INPUT=$(echo -n "un string no muy largo" | base64)
# Operación de firma
SIGNATURE=$(vault write -format=json transit/sign/signature input=$INPUT hash_algorithm=sha2-256 -format=json | jq -r .data.signature)
echo $SIGNATURE

# Verificación de firma
vault write transit/verify/signature input=$INPUT signature=$SIGNATURE  hash_algorithm=sha2-256

vault:v1:bEyZd4Y4JGGXFu82QJHsM3o2KsrNsIkUc8//Yu1ud027ZQkzAMsc9O6W1i47talHcaKOm9ZKfdkbVmSFScabN3kK/HeHCmqwjOGwhq3NN2F0rQevZuOZ2lEYwUH9UmEoSEwmXfBU63AiXEaUpSggULu10yGzY33c5W0DFpwKgrOE70bF7gXlHTxPO8uXn6pRnX+NMZ2PFfP3G6P0VtGrhcZ2LnPOplYWyEqKOIxrPUm7gtFY4axMCsrXHzUYwrS1m0PTT2aFnOohEABFVWQUhuI0IhiFT1enCAhjzJNDXwhaYLUvso+J0u13dgSIXxKQZqxUk8Z52hC7G3U0tLuEC7OsVsuLSyoCSp6IikKL99pmw5VCrh0WjnzKs0lyO2UFjoUqfHZq4zSSuZ4gzty6MVh5GtOzVZjuuBs6QQsWNBABYCaFNCInoGo/HO2A2O2V1fWKeq8a5OUVq0QyBXT23Nj/Mr55FaLfcAnQ+zw3Hgq806/i1WYWEhUFbBhU73sjShVVfYgbFKx4fn4p5X9HsYSoqubAH8Evi2cGX8Di1wwhhBl9ysVo1bhiqXg+1pWI6nHfuf8DRBjIic0wMdo9c41b82TZfdcRqTnBe+7ZCD+caZZUGyoLxLHPSIidilzl+a7ZcFcNMIFG1FZSO8Z2Yr98QHGCTM68drOeB5yNltc=
Key      Value
---      -----
valid    true


## HMAC

In [10]:
%%bash
vault write -f  transit/keys/hmacker type=hmac key_size=512

Key                       Value
---                       -----
allow_plaintext_backup    false
auto_rotate_period        0s
deletion_allowed          false
derived                   false
exportable                false
imported_key              false
key_size                  512
latest_version            1
min_available_version     0
min_decryption_version    1
min_encryption_version    0
name                      hmacker
supports_decryption       false
supports_derivation       false
supports_encryption       false
supports_signing          false
type                      hmac


In [11]:
%%bash

INPUT=$(echo -n "otro string largo, pero que muy largo...... ... .. " | base64)
# Operación de hmac
HMAC=$(vault write -format="json" transit/hmac/hmacker algorithm="sha2-512" input=$INPUT  | jq -r .data.hmac)
echo $HMAC

# Verificación de hmac
vault write transit/verify/hmacker input=$INPUT algorithm="sha2-512" hmac=$HMAC

#vault:v1:iLXnIDfw+t+C1JKAZlO9xOIkbaS1qJfmZh1knNeQ3BTov735obcXob4s0rqSdrXz4hCO1eGa61Rc7raoI6mrUA==

vault:v1:LmYcNbiHUBOpm9c93L76P1cVsp/Yq0pZI61SRMudPwBFI9mLZCk1+ex6gdNsVl+cEdmvPn1Fn+fZc+A7mLAMPg==
Key      Value
---      -----
valid    true


## HASH

In [12]:
%%bash
INPUT=$( echo -n "otro string largo pero que muy largo 32323232323" | base64)

# Operación de hash
HASH=$(vault write -format=json transit/hash input=$INPUT algorithm=sha3-512 format=base64 -format=json | jq -r .data.sum)
echo $HASH

tjwNxF5LBJsuBTPjfbLEWiF2bYMabF4LGF8Qx95RQue5W6DpfpbI3Mobg1aNlSLhgiyIfVrfcN8or9rJLcS+AQ==
